In [1]:
!pip3 install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=540bbc6dad4ea2260247d1627b5a6a2f46c958515ad8fd7d2dc53fa65bbdd01d
  Stored in directory: /Users/being/Library/Caches/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import evaluate

/Users/being/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
import evaluate
import pandas as pd
import os

class TranslationEvaluator:
    def __init__(self, predictions, references, output_path='evaluation_results.csv'):
        self.predictions = predictions
        self.references = references
        self.output_path = output_path
        self._validate_inputs()
        self.metrics = {
            'bleu': evaluate.load('bleu'),
            'meteor': evaluate.load('meteor'),
            'ter': evaluate.load('ter'),
            'rouge': evaluate.load('rouge')
        }
        self.results = []

    def _validate_inputs(self):
        if not isinstance(self.predictions, list) or not isinstance(self.references, list):
            raise ValueError("Predictions and references should be lists.")
        if len(self.predictions) != len(self.references):
            raise ValueError("Predictions and references must have the same length.")
        ref_len = len(self.references[0])
        for ref in self.references:
            if len(ref) != ref_len:
                raise ValueError("Each reference list must have the same number of sentences.")

    def compute_scores(self):
        for idx, (pred, ref) in enumerate(zip(self.predictions, self.references)):
            row = {
                'index': idx,
                'sentence_references': ' ||| '.join(ref),
                'sentence_predictions': pred
            }
            for metric_name, metric in self.metrics.items():
                score = metric.compute(predictions=[pred], references=[ref])
                if metric_name == 'rouge':
                    row.update({f'Metric_{metric_name}_{k}': v for k, v in score.items()})
                elif metric_name == 'ter':
                    row[f'Metric_{metric_name}'] = score['score']
                else:
                    row[f'Metric_{metric_name}'] = score[metric_name]
            self.results.append(row)

    def save_to_csv(self):
        df = pd.DataFrame(self.results)
        df.to_csv(self.output_path, index=False)
        print(f"Results saved to {os.path.abspath(self.output_path)}")

# 예시 데이터
references = [
    ["이것은 예시 번역입니다.", "이것은 또 다른 예시입니다."],
    ["이것은 또 다른 예시입니다.", "또 다른 예시 번역입니다."]
]


predictions = [
    "이것은 예시 번역입니다.",
    "또 다른 예시 번역입니다."
]



# 클래스 사용 예시
evaluator = TranslationEvaluator(predictions, references)
evaluator.compute_scores()
evaluator.save_to_csv()

[nltk_data] Downloading package wordnet to /Users/being/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/being/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/being/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Results saved to /Users/being/Desktop/workspace2024/TranslationEvaluator/src/evaluation_results.csv
